In [1]:
import pandas as pd
import syft as sy

sy.logger.remove()

In [2]:
ca_domain_node = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)
it_domain_node = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8082)

Connecting to http://localhost:8081... done! 	 Logging into canada... done!
Connecting to http://localhost:8082... done! 	 Logging into italy... done!


In [3]:
ca_domain_node.store.pandas[ca_domain_node.store.pandas["object_type"] == "<class 'pandas.core.frame.DataFrame'>"]

,ID,Tags,Description,object_type
61,<UID: 30af0e7bb8a14a55975577bccc74b617>,[#Canada-feb2020],,<class 'pandas.core.frame.DataFrame'>
70,<UID: 5ac13049f21d4ea293a05fed52dfe613>,[#Canada-feb2020],,<class 'pandas.core.frame.DataFrame'>
75,<UID: 513a4da252d54feda9079da6e02ec907>,[#Canada-feb2020],,<class 'pandas.core.frame.DataFrame'>
79,<UID: 0017f4b19fd148cd832733ade01a386a>,[#feb2020],,<class 'pandas.core.frame.DataFrame'>
82,<UID: f0e41f9923af4e2385a3a5951fb51905>,[#40k-feb2020],,<class 'pandas.core.frame.DataFrame'>
102,<UID: af3f9215955b4b3685a9daa3748c1add>,[#feb2020],,<class 'pandas.core.frame.DataFrame'>
103,<UID: 9ec35de9905449cdb0fb83f65fa35f24>,[#feb2020],,<class 'pandas.core.frame.DataFrame'>
128,<UID: 440b97516b5f48f0869b965dac2d8d9e>,[#feb2020],,<class 'pandas.core.frame.DataFrame'>
129,<UID: f5b077365cc24e5db130326d39edae46>,[#feb2020],,<class 'pandas.core.frame.DataFrame'>
130,<UID: 3ea6c3b10bec4dd2bd96b114dba36aea>,[#feb2020],,<class 'pandas.core.frame.DataFrame'>


In [4]:
it_domain_node.store.pandas

,ID,Tags,Description,object_type
0,<UID: 5d57bbff0b5b4b31abcd57c65f4bb11f>,[#Italy-feb2020],,<class 'pandas.core.frame.DataFrame'>
1,<UID: d48655485cc24216bd914264cbcb9c40>,[],,<class 'syft.lib.python.Int'>
2,<UID: 85316e0c26e04dc68d94e25a5fe092e0>,[],,<class 'syft.lib.python.Int'>
3,<UID: f54a3c47f2984458b7a18d8296128e83>,[],,<class 'syft.lib.python._SyNone'>
4,<UID: 9f8dbd2e51884953bde3e8e11a351224>,[],,<class 'syft.lib.python.Tuple'>
...,...,...,...,...
80,<UID: 096c19a1a1054894a1640c3ad8e72756>,[],,<class 'syft.lib.python.String'>
81,<UID: d56d537a88e94129ba674f42db628406>,[],,<class 'numpy.ndarray'>
82,<UID: 3b6293251e8c41a7839b61bb49afc29a>,[],,<class 'syft.lib.python.String'>
83,<UID: 44c9c2cbc5bf4b6dbf69cdd6a6dd79c2>,[],,<class 'numpy.ndarray'>


In [13]:
ca_dataset_ptr = ca_domain_node.store[89]
it_dataset_ptr = it_domain_node.store[0]

In [14]:
ca_dataset_ptr.tags, it_dataset_ptr.tags

(['#40k-feb2020-numpy'], ['#Italy-feb2020'])

In [8]:
from syft.core.tensor.smpc.mpc_tensor import MPCTensor

In [9]:
parties = [ca_domain_node, it_domain_node]

In [32]:
mpc_tensor = MPCTensor(secret=ca_dataset_ptr, shape=(40000, 3), parties=parties)

In [33]:
r = 2 * mpc_tensor

In [36]:
r.reconstruct()

UnknownPrivateException: UnknownPrivateException has been triggered.

In [179]:
import numpy as np

In [231]:
e = np.ones(10)
e = e.astype(int)

t = mpc_tensor + e

In [233]:
s = t.reconstruct()

array([   9286,  116605, 1495176,    2249,   47841,    3527,    5463,
        311426,   11787,   33716])